In [ ]:
!pip install MetaTrader5

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import datetime, timezone
import time
import MetaTrader5 as mt5

In [ ]:
class Modify:
    def __init__(self, symbol, lot, sl, tp, deviation, tsl):
        self.symbols = symbol
        self.lot = lot
        self.sl = sl #pips
        self.tp = tp #pips
        self.deviation = deviation
        self.tsl = tsl
        self.raw = {}

    def calculate_profit(self):
        profits = {}
        for sym in self.symbols:
            positions = mt5.positions_get(symbol=sym)

            if positions:
                df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())
                if not df.empty:
                    df['time'] = pd.to_datetime(df['time'], unit='s')
                    profit = float(df['profit'].sum())
                    profits[sym] = profit
                    self.trailing_stoploss()

                self.raw[sym] = df

        return profits
    
        
    def reverse_type(order_type):
        if order_type == mt5.ORDER_TYPE_BUY:
            return mt5.ORDER_TYPE_SELL
        
        elif order_type == mt5.ORDER_TYPE_SELL:
            return mt5.ORDER_TYPE_BUY
        
    def get_close_price(symbol, order_type):
        if order_type == mt5.ORDER_TYPE_BUY:
            return mt5.symbol_info(symbol).ask
        
        elif order_type == mt5.ORDER_TYPE_SELL:
            return mt5.symbol_info(symbol).bid
    
    def closing_order(self, symbol, order_type, get_close_price)
        request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": pos1.ticket,
                        "symbol": pos1.symbol,
                        "volume": pos1.volume,
                        "type": reverse_type(pos1.type),
                        "price": get_close_price(pos1.symbol, pos1.type),
                        "deviation": 500,
                        "magic": 10,
                        "comment": "python close order",
                        "type_time": mt5.ORDER_TIME_GTC,
                        "type_filling": mt5.ORDER_FILLING_IOC,
                    }

        result_modify = mt5.order_send(request)
        result_modify
        print(result_modify)

    def open_order_modify_buy(self, symbol, symbol_data):
        request_modify_buy = {
            "action": mt5.TRADE_ACTION_SLTP,
            "position": symbol_data['ticket'].iloc[0],
            "price": mt5.symbol_info_tick(symbol).ask,
            "sl": mt5.symbol_info_tick(symbol).ask - (self.sl * 10) * mt5.symbol_info(symbol).point,
            "tp": mt5.symbol_info_tick(symbol).ask + (self.tp * 10) * mt5.symbol_info(symbol).point,
            "comment": "Trailing Buys",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result_modify_buy = mt5.order_send(request_modify_buy)
        print(result_modify_buy)

    def open_order_modify_sell(self, symbol, symbol_data):
        request_modify_sell = {
            "action": mt5.TRADE_ACTION_SLTP,
            "position": symbol_data['ticket'].iloc[0],
            "price": mt5.symbol_info_tick(symbol).bid,
            "sl": mt5.symbol_info_tick(symbol).bid + (self.sl * 10) * mt5.symbol_info(symbol).point,
            "tp": mt5.symbol_info_tick(symbol).bid - (self.tp * 10) * mt5.symbol_info(symbol).point,
            "comment": "Trailing Sells",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result_modify_sell = mt5.order_send(request_modify_sell)
        print(result_modify_sell)